## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [3]:
# Se establece qué columnas se eliminan

# col_eliminar_creditos = []
col_eliminar_tarjetas = ['nivel_tarjeta']

# Se ejecuta la operación

# df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [4]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,24.0,2.0


Limpieza de los datos (filtrado a nivel de filas)

In [5]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok


In [6]:
# Definir la función para la Regla 2
def regla_duracion_minima_credito(row, duracion_minima):
    duracion_credito = row.duracion_credito
    pct_ingreso = row.pct_ingreso
    situacion_vivienda = row.situacion_vivienda  # Verificar si la situación de vivienda es 'propia'
    
    if duracion_credito == duracion_minima:
        if pct_ingreso > 0.6 and situacion_vivienda != 'PROPIA':
            # Es un error, no cumple la regla definida
            return 'err'
        else:
            return 'ok'
    else:
        return 'ok'

# Aplicar la función para todos los elementos del dataset
duracion_minima_permitida = 2  # Supón que la duración mínima permitida es 2
regla_duracion_minima = df_creditos.apply(lambda row: regla_duracion_minima_credito(row, duracion_minima_permitida), axis=1).rename("regla_duracion_minima")

# Unir los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_duracion_minima], axis=1)

# Visualizar los datos que no cumplen con la Regla 2
print("Se visualizan las tuplas que no cumplen con la Regla 2:")
display(df_creditos[df_creditos.regla_duracion_minima == 'err'])



Se visualizan las tuplas que no cumplen con la Regla 2:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,regla_duracion_minima
36,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,1,N,ok,err
571,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,1,N,err,err
577,716027208.0,26,20050,2,10.0,ALQUILER,26000,EDUCACIÓN,0.77,9.76,1,N,ok,err
587,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,1,N,ok,err
3535,717886008.0,24,25000,2,4.0,HIPOTECA,37000,SALUD,0.68,10.08,0,N,ok,err
6758,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,1,N,err,err
7220,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,1,Y,ok,err


In [7]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]
temp_2 = temp[temp['antiguedad_empleado'] < 100]
temp_3=temp[temp['situacion_vivienda'] != 'Propia']
# Otro filtro posible: por la regla de negocio agregada

temp_c = temp_2[temp_2['regla_pct_ingresos'] == 'ok']
temp_d=temp_3[temp_3['regla_duracion_minima'] == 'ok']

print(f"Filas después del filtro: {temp_d.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10116


In [8]:
# Imprimir las filas antes del filtro
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

# Aplicar los filtros secuencialmente
temp = df_creditos[
    (df_creditos['edad'] < 90) &
    (df_creditos['antiguedad_empleado'] < 100) &
    (df_creditos['situacion_vivienda'] != 'Propia') &
    (df_creditos['regla_pct_ingresos'] == 'ok') &
    (df_creditos['regla_duracion_minima'] == 'ok')
]

# Imprimir las filas después de aplicar todos los filtros
print(f"Filas después del filtro: {temp.shape[0]}")


Filas antes del filtro: 10127
Filas después del filtro: 9765


Integración de datos

In [9]:
df_integrado = pd.merge(temp, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 9765


In [10]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 23


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [11]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

# Sobre este resultado será necesario eliminar las columnas auxiliares
df_final = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_final.head()


,estado_civil_N,estado_credito_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,...,regla_duracion_minima,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,C,P,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,...,ok,39.0,CASADO,ACTIVO,1144.0,M,12691.0,SECUNDARIO_COMPLETO,42.0,3.0
1,C,C,713982108.0,23,35000,2,4.0,ALQUILER,65500,SALUD,...,ok,36.0,CASADO,ACTIVO,1887.0,M,3418.0,UNIVERSITARIO_COMPLETO,20.0,3.0
2,C,C,710821833.0,24,35000,4,8.0,ALQUILER,54400,SALUD,...,ok,54.0,CASADO,ACTIVO,1314.0,M,9095.0,DESCONOCIDO,26.0,1.0
3,N,C,769911858.0,21,2500,2,2.0,PROPIA,9900,INVERSIONES,...,ok,34.0,DESCONOCIDO,ACTIVO,1171.0,F,3313.0,SECUNDARIO_COMPLETO,20.0,4.0
4,C,C,709106358.0,26,35000,3,8.0,ALQUILER,77100,EDUCACIÓN,...,ok,21.0,CASADO,ACTIVO,816.0,M,4716.0,UNIVERSITARIO_INCOMPLETO,28.0,3.0


Atributos numéricos que se discretizan

In [12]:
df_integrado['operaciones_ult_12m'].describe()

count    9765.00000
mean       64.98382
std        23.52001
min        10.00000
25%        45.00000
50%        67.00000
75%        81.00000
max       139.00000
Name: operaciones_ult_12m, dtype: float64

In [13]:
# Antiguedad del empleado

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(df_integrado['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50_a_75      3269
75_a_100     2915
30_a_50      2320
mayor_100     670
15_a_30       567
menor_15       24
Name: count, dtype: int64

In [14]:
df_final.columns

Index(['estado_civil_N', 'estado_credito_N', 'id_cliente', 'edad',
       'importe_solicitado', 'duracion_credito', 'antiguedad_empleado',
       'situacion_vivienda', 'ingresos', 'objetivo_credito', 'pct_ingreso',
       'tasa_interes', 'estado_credito', 'falta_pago', 'regla_pct_ingresos',
       'regla_duracion_minima', 'antiguedad_cliente', 'estado_civil',
       'estado_cliente', 'gastos_ult_12m', 'genero', 'limite_credito_tc',
       'nivel_educativo', 'operaciones_ult_12m', 'personas_a_cargo'],
      dtype='object')

In [15]:
# Agregar en este listado otros atributos que pudieran discretizarse o transformarse
col_eliminar_final = [
              'edad',
              'estado_civil',
              'estado_credito',
              'antiguedad_empleado',
              'antiguedad_cliente', 
              'ingresos',
              'pct_ingreso', 
              'tasa_interes',
              'regla_pct_ingresos',
              'gastos_ult_12m', 
              'limite_credito_tc', 
              'operaciones_ult_12m',
              'id_cliente']

df_final.drop(col_eliminar_final, inplace=True, axis=1)

df_final = pd.concat([operaciones_ult_12m_N, 
                      gastos_ult_12m_N, 
                      limite_credito_tc_N, 
                      antiguedad_cliente_N, 
                      tasa_interes_N, 
                      ingresos_N, 
                      pct_ingreso_N, 
                      antiguedad_empleados_N, 
                      edad_N, 
                      df_final], axis=1)
df_final.head(25)


,operaciones_ult_12m,gastos_ult_12m,limite_credito_tc,antiguedad_cliente,tasa_interes,ingresos,pct_ingreso,antiguedad_empleado,edad,estado_civil_N,...,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,falta_pago,regla_duracion_minima,estado_cliente,genero,nivel_educativo,personas_a_cargo
0,30_a_50,2k_a_4k,mayor_10k,2y_a_4y,7p_a_15p,hasta_20k,hasta_20,5_a_10,menor_25,C,...,1000,2,PROPIA,EDUCACIÓN,N,ok,ACTIVO,M,SECUNDARIO_COMPLETO,3.0
1,15_a_30,2k_a_4k,3k_a_5k,2y_a_4y,15p_a_20p,50k_a_100k,40_a_60,5_a_10,menor_25,C,...,35000,2,ALQUILER,SALUD,N,ok,ACTIVO,M,UNIVERSITARIO_COMPLETO,3.0
2,15_a_30,2k_a_4k,5k_a_10k,mayor_4y,7p_a_15p,50k_a_100k,40_a_60,5_a_10,menor_25,C,...,35000,4,ALQUILER,SALUD,Y,ok,ACTIVO,M,DESCONOCIDO,1.0
3,15_a_30,2k_a_4k,3k_a_5k,2y_a_4y,7p_a_15p,hasta_20k,20_a_40,menor_5,menor_25,N,...,2500,2,PROPIA,INVERSIONES,N,ok,ACTIVO,F,SECUNDARIO_COMPLETO,4.0
4,15_a_30,menor_1k,3k_a_5k,menor_2y,7p_a_15p,50k_a_100k,40_a_60,5_a_10,25_a_30,C,...,35000,3,ALQUILER,EDUCACIÓN,N,ok,ACTIVO,M,UNIVERSITARIO_INCOMPLETO,3.0
5,30_a_50,2k_a_4k,mayor_10k,2y_a_4y,7p_a_15p,50k_a_100k,40_a_60,5_a_10,menor_25,C,...,35000,4,ALQUILER,SALUD,N,ok,ACTIVO,M,DESCONOCIDO,4.0
6,30_a_50,2k_a_4k,mayor_10k,2y_a_4y,7p_a_15p,50k_a_100k,40_a_60,5_a_10,menor_25,N,...,35000,2,ALQUILER,PERSONAL,N,ok,ACTIVO,M,SECUNDARIO_COMPLETO,0.0
7,15_a_30,2k_a_4k,mayor_10k,2y_a_4y,7p_a_15p,hasta_20k,hasta_20,5_a_10,menor_25,S,...,1600,3,PROPIA,INVERSIONES,N,ok,ACTIVO,M,UNIVERSITARIO_INCOMPLETO,3.0
8,30_a_50,2k_a_4k,mayor_10k,2y_a_4y,7p_a_15p,50k_a_100k,40_a_60,5_a_10,menor_25,S,...,35000,4,ALQUILER,INVERSIONES,N,ok,ACTIVO,M,UNIVERSITARIO_COMPLETO,2.0
9,30_a_50,2k_a_4k,5k_a_10k,2y_a_4y,7p_a_15p,hasta_20k,40_a_60,menor_5,menor_25,N,...,4500,2,PROPIA,MEJORAS_HOGAR,N,ok,ACTIVO,M,UNIVERSITARIO_INCOMPLETO,5.0


In [16]:
df_final.columns

Index(['operaciones_ult_12m', 'gastos_ult_12m', 'limite_credito_tc',
       'antiguedad_cliente', 'tasa_interes', 'ingresos', 'pct_ingreso',
       'antiguedad_empleado', 'edad', 'estado_civil_N', 'estado_credito_N',
       'importe_solicitado', 'duracion_credito', 'situacion_vivienda',
       'objetivo_credito', 'falta_pago', 'regla_duracion_minima',
       'estado_cliente', 'genero', 'nivel_educativo', 'personas_a_cargo'],
      dtype='object')

Exportación de metadatos

In [17]:
import dtale as dt

informe2 = dt.show(df_final)
informe2.open_browser()

In [18]:
print(f"Cantidad de columnas del dataset final: {df_final.shape[1]}")
print(f"Cantidad de filas del dataset final: {df_final.shape[0]}")

Cantidad de columnas del dataset final: 21
Cantidad de filas del dataset final: 9765


Exportación del dataset

In [19]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)

In [20]:
import pandas as pd
import dtale

# Cargar el archivo CSV en un DataFrame de Pandas
df = pd.read_csv("../../data/final/datos_finales.csv", sep=';')

# Iniciar D-Tale con el DataFrame
dtale.show(df).open_browser()